In [1]:
import math

## Q3

In [2]:
def initialize():
    x = [1., -1.]
    w = [[1.,1.,1.], [-1.,-1.,-1.]]
    b = [0.,0.,0.]
    k = [0.,0.,0.]
    h = [0.,0.,0.]
    v = [[1.,1.],[-1.,-1.],[-1.,-1.]]
    c = [0., 0.]
    t = [1., 0.]
    s = [0., 0.]
    y = []
    l = 0
    return x,w,b,k,h,v,c,t,s,y,l


def sigmoid(x):
    return 1. / (1. + math.exp(-x))

def softmax(x):
    sum_exp = 0
    for i in x:
        sum_exp += math.exp(i)
        return math.exp(i)/ sum_exp

def forward_pass(x,w,b,k,h,v,c,t,s,y,loss):
    #Forward pass:
    for lin in range(3): # k nodes len(k)
        for i in range(2): #input nodes
            k[lin] += w[i][lin] * x[i]
        k[lin] += b[lin]
    #print(k)

    for hid in range(3):
        h[hid] = sigmoid(k[hid]) # 1. / (1. + math.exp(-k[hid])) 
    #print(h)

    for soft in range(len(s)):
        for hi in range(len(h)):
            s[soft] += v[hi][soft] * h[hi] #hidden * weights
        s[soft] += c[soft] #second bias
    #print(s)

    sum_exp = 0
    for i in s:
        sum_exp += math.exp(i)

    for i in range(len(s)):  
        y.append(math.exp(s[i])/ sum_exp)
    #print(y)

    for node in range(len(y)):
        if t[node] == 1.: l = (- math.log(y[node]))

    return x,w,b,k,h,v,c,t,s,y,l
    #correct output (t), the loss (l), and all intermediate values (return k, h, o(s from softmax), y, l) 

x,w,b,k,h,v,c,t,s,y,l = initialize()
x,w,b,k,h,v,c,t,s,y,l = forward_pass(x,w,b,k,h,v,c,t,s,y,l)
h, v, l, s, y, k

([0.8807970779778823, 0.8807970779778823, 0.8807970779778823],
 [[1.0, 1.0], [-1.0, -1.0], [-1.0, -1.0]],
 0.6931471805599453,
 [-0.8807970779778823, -0.8807970779778823],
 [0.5, 0.5],
 [2.0, 2.0, 2.0])

In [3]:
def backward_pass(x,w,b,k,h,v,s,y):
    dk = k
    dh = h
    dv = v
    dw = w
    db = b
    dy = y
    
    for elem in range(len(y)):
        if elem != 1:
            dy[elem] = y[elem] - 1
    print(dy)

    for soft in range(len(s)):
        for hi in range(len(h)): #sigmoid hidden layer
            dv[hi][soft] = dy[soft] * h[hi]

    for soft in range(len(s)):
        for hi in range(len(h)): #sigmoid hidden layer
            dh[hi] = dy[soft] * v[hi][soft]

    dc = dy

    for lin in range(len(dk)):     
        dk[lin] = dh[lin] * h[lin] * (1-h[lin])


    for lin in range(len(k)):
        for inp in range(len(x)):
            dw[inp][lin] = dk[lin] * x[inp]     
    
    db[lin] = dk[lin] 

    return dw, db, dv, dc, dy

dw, db, dv, dc, dy = backward_pass(x,w,b,k,h,v,s,y)

[-0.5, 0.5]


## Q4

In [4]:
import numpy as np
from urllib import request
import gzip
import pickle
import os
import matplotlib.pyplot as plt

In [5]:
def load_synth(num_train=60_000, num_val=10_000, seed=0):
    """
    Load some very basic synthetic data that should be easy to classify. Two features, so that we can plot the
    decision boundary (which is an ellipse in the feature space).
    :param num_train: Number of training instances
    :param num_val: Number of test/validation instances
    :param num_features: Number of features per instance
    :return: Two tuples and an integer: (xtrain, ytrain), (xval, yval), num_cls. The first contains a matrix of training
     data with 2 features as a numpy floating point array, and the corresponding classification labels as a numpy
     integer array. The second contains the test/validation data in the same format. The last integer contains the
     number of classes (this is always 2 for this function).
    """
    np.random.seed(seed)

    THRESHOLD = 0.6
    quad = np.asarray([[1, -0.05], [1, .4]])

    ntotal = num_train + num_val

    x = np.random.randn(ntotal, 2)

    # compute the quadratic form
    q = np.einsum('bf, fk, bk -> b', x, quad, x)
    y = (q > THRESHOLD).astype(np.int)

    return (x[:num_train, :], y[:num_train]), (x[num_train:, :], y[num_train:]), 2

def load_mnist(final=False, flatten=True):
    """
    Load the MNIST data.
    :param final: If true, return the canonical test/train split. If false, split some validation data from the training
       data and keep the test data hidden.
    :param flatten: If true, each instance is flattened into a vector, so that the data is returns as a matrix with 768
        columns. If false, the data is returned as a 3-tensor preserving each image as a matrix.
    :return: Two tuples and an integer: (xtrain, ytrain), (xval, yval), num_cls. The first contains a matrix of training
     data and the corresponding classification labels as a numpy integer array. The second contains the test/validation
     data in the same format. The last integer contains the number of classes (this is always 2 for this function).
     """

    if not os.path.isfile('mnist.pkl'):
        init()

    xtrain, ytrain, xtest, ytest = load()
    xtl, xsl = xtrain.shape[0], xtest.shape[0]

    if flatten:
        xtrain = xtrain.reshape(xtl, -1)
        xtest  = xtest.reshape(xsl, -1)

    if not final: # return the flattened images
        return (xtrain[:-5000], ytrain[:-5000]), (xtrain[-5000:], ytrain[-5000:]), 10

    return (xtrain, ytrain), (xtest, ytest), 10

# Numpy-only MNIST loader. Courtesy of Hyeonseok Jung
# https://github.com/hsjeong5/MNIST-for-Numpy

filename = [
["training_images","train-images-idx3-ubyte.gz"],
["test_images","t10k-images-idx3-ubyte.gz"],
["training_labels","train-labels-idx1-ubyte.gz"],
["test_labels","t10k-labels-idx1-ubyte.gz"]
]

def download_mnist():
    base_url = "http://yann.lecun.com/exdb/mnist/"
    for name in filename:
        print("Downloading "+name[1]+"...")
        request.urlretrieve(base_url+name[1], name[1])
    print("Download complete.")

def save_mnist():
    mnist = {}
    for name in filename[:2]:
        with gzip.open(name[1], 'rb') as f:
            mnist[name[0]] = np.frombuffer(f.read(), np.uint8, offset=16).reshape(-1,28*28)
    for name in filename[-2:]:
        with gzip.open(name[1], 'rb') as f:
            mnist[name[0]] = np.frombuffer(f.read(), np.uint8, offset=8)
    with open("mnist.pkl", 'wb') as f:
        pickle.dump(mnist,f)
    print("Save complete.")

def init():
    download_mnist()
    save_mnist()

def load():
    with open("mnist.pkl",'rb') as f:
        mnist = pickle.load(f)
    return mnist["training_images"], mnist["training_labels"], mnist["test_images"], mnist["test_labels"]

In [6]:
(xtrain, ytrain), (xval, yval), num_cls = load_synth()

Implement a training loop for your network and show that the training loss drops as training progresses.
Some tips:
- How you initialize the weights is an important choice. For now, you can set the regular weights to some normally distributed random value, and the bias weights to 0. We'll delve into this question more in later lectures.
- You can use the python random package to generate normally distributed random values, or generate some values online and hardcode them.
- Our data loaders provide the target classes as integer values. It's easiest to work out the derivative in terms of these values directly, but you can also convert them to one-hot vectors as shown in the image.
- You can use stochastic gradient descent, calculating the loss over one instance at a time.

In [27]:
def initialize_random():
    w = np.random.randn(2,3)# [[1.,1.,1.], [-1.,-1.,-1.]]
    v = np.random.randn(3,2) # [[1.,1.],[-1.,-1.],[-1.,-1.]]

    b = [0.,0.,0.]
    c = [0., 0.]

    k = [0.,0.,0.]
    h = [0.,0.,0.]
    s = [0., 0.]
    y = []
    l = 0
    
    return w,b,k,h,v,c,s,y,l
    

In [28]:
def st_grad_des(alpha, xtrain, ytrain):
    loss = []
    w,b,k,h,v,c,s,y,l = initialize_random()    
    for i in range(1):
        for x, t in zip(xtrain, ytrain): # for each input x and target t in data
            x,w,b,k,h,v,c,t,s,y,l = forward_pass(x,w,b,k,h,v,c,t,s,y,l)
            # dw, db, dv, dc, dy = backward_pass(x,w,b,k,h,v,s,y)
            # w = w - alpha * dw #derivative of loss of x and t in data
            # b = b - alpha * db
            # v = v - alpha * dv
            # c = c - alpha * dc
            # y = y - alpha * dy
        # plt.plot(i,l, 'o')

st_grad_des(1,xtrain, ytrain)

IndexError: invalid index to scalar variable.